# Import libraries

In [93]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)



In [94]:
BI_FOLDER_PATH = "bi_outputs/"

# connect to postgresql

In [95]:
load_dotenv()

def get_engine():
    """
    Returns a reusable SQLAlchemy engine for PostgreSQL.
    Credentials are read from .env file.
    """
    user = os.getenv("PG_USER")
    host = os.getenv("PG_HOST")
    password = os.getenv("PG_PASSWORD")
    port = os.getenv("PG_PORT", 5433)
    db = os.getenv("PG_DB")

    url = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}"

    engine = create_engine(url)
    return engine

def run_query(sql: str) -> pd.DataFrame:
    """
    Executes a SQL query and returns a Pandas DataFrame
    """
    engine = get_engine()
    df = pd.read_sql(sql, engine)
    return df

# Seller performance table

In [96]:
df_sellers_performance = run_query(
    """
        --- sellers that contribute the most to GMV
        --- EXPLAIN ANALYZE
        WITH order_metrics AS (
            SELECT 
                oi.seller_id,
                COUNT(DISTINCT o.order_id) AS total_order_count,                                                                
                COUNT(DISTINCT CASE 
                    WHEN o.order_status = 'delivered' 
                    THEN o.order_id 
                END) AS total_delivered_order_count,                      
                COUNT(DISTINCT oi.order_item_id) AS distinct_total_order_item_count,                                                               
                COUNT(DISTINCT CASE 
                    WHEN o.order_status = 'delivered' 
                    THEN oi.order_item_id 
                END) AS distinct_delivered_order_item_count,      
                SUM(CASE 
                    WHEN o.order_status = 'delivered' 
                    THEN oi.price
                    ELSE 0 
                END) AS delivered_items_gmv,
                COUNT(DISTINCT CASE 
                    WHEN o.order_status = 'canceled' 
                    THEN o.order_id 
                END) AS total_order_cancel_count,
                COUNT(DISTINCT CASE 
                    WHEN o.order_status = 'canceled' 
                    THEN oi.order_item_id 
                END) AS distinct_order_items_cancel_count
            FROM order_items oi
            INNER JOIN orders o 
                ON oi.order_id = o.order_id
            GROUP BY oi.seller_id
        )

        SELECT 
            s.seller_id,
            om.total_order_count,
            om.total_delivered_order_count,
            om.distinct_total_order_item_count,
            om.distinct_delivered_order_item_count,
            om.delivered_items_gmv,
            om.total_order_cancel_count,
            om.distinct_order_items_cancel_count
        FROM sellers s
        INNER JOIN order_metrics om ON s.seller_id = om.seller_id
    """
)

df_sellers_performance.head()

,seller_id,total_order_count,total_delivered_order_count,distinct_total_order_item_count,distinct_delivered_order_item_count,delivered_items_gmv,total_order_cancel_count,distinct_order_items_cancel_count
0,0015a82c2db000af6aaaf3ae2ecb0532,3,3,1,1,2685.00,0,0
1,001cca7ae9ae17fb1caed9dfb1094831,200,195,5,5,24487.03,0,0
2,001e6ad469a905060d959994f1b41e4f,1,0,1,0,0.00,1,1
3,002100f778ceb8431b7a1020ff7ab48f,51,50,2,2,1216.60,0,0
4,003554e2dce176b5555353e4f3555ac8,1,1,1,1,120.00,0,0


In [97]:
df_sellers_performance.to_csv(f'{BI_FOLDER_PATH}bi_seller_performance.csv', index=False)

# Product category Performance Table

In [98]:
df_products_performance = run_query(
    """
    WITH category_sales AS (
        SELECT
                pt.product_category_name_english AS product_category_name,
                COUNT(DISTINCT CASE 
                    WHEN o.order_status = 'delivered' 
                    THEN oi.order_item_id 
                END) AS total_order_item_count,                                                                                                                           -- total unique orders items sold per product
                SUM(CASE WHEN 
                    o.order_status = 'delivered' 
                    THEN oi.price 
                    ELSE 0 
                END) AS delivered_items_gmv,             
                COUNT(DISTINCT CASE 
                    WHEN o.order_status = 'canceled' 
                    THEN oi.order_item_id 
                END) AS order_items_cancel_count,                                                                                                                    -- total unique canceled order items per product
                COUNT(DISTINCT CASE 
                    WHEN o.order_status = 'canceled' 
                    THEN o.order_id 
                END) AS canceled_orders_count
            FROM orders o
            INNER JOIN order_items oi ON o.order_id = oi.order_id
            INNER JOIN products p ON oi.product_id = p.product_id
            INNER JOIN product_category_name_translation pt ON p.translation_id = pt.translation_id
            GROUP BY pt.product_category_name_english
    ),

    customer_category_orders AS (
        SELECT 
            c.customer_unique_id,
            pt.product_category_name_english AS product_category_name,
            COUNT(DISTINCT o.order_id) AS num_orders
        FROM orders o
        JOIN order_items oi ON o.order_id = oi.order_id
        JOIN products p ON oi.product_id = p.product_id
        JOIN customers c ON o.customer_id = c.customer_id
        JOIN product_category_name_translation pt ON p.translation_id = pt.translation_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_unique_id, pt.product_category_name_english
    ),

    category_repeat_rate AS (
        SELECT
            product_category_name,
            COUNT(customer_unique_id) AS total_customers,
            COUNT(CASE WHEN num_orders > 1 THEN 1 END) AS repeat_customers,
            ROUND(
                COUNT(CASE WHEN num_orders > 1 THEN 1 END)::numeric
                / COUNT(customer_unique_id), 2
            ) AS repeat_purchase_rate
        FROM customer_category_orders
        GROUP BY product_category_name
    )

    SELECT
        s.product_category_name,
        s.total_order_item_count,
        s.delivered_items_gmv,
        s.order_items_cancel_count,
        s.canceled_orders_count,
        r.total_customers,
        r.repeat_customers,
        r.repeat_purchase_rate
    FROM category_sales s
    LEFT JOIN category_repeat_rate r
        ON s.product_category_name = r.product_category_name
    ORDER BY s.delivered_items_gmv DESC;

    """
)

df_products_performance.head()


,product_category_name,total_order_item_count,delivered_items_gmv,order_items_cancel_count,canceled_orders_count,total_customers,repeat_customers,repeat_purchase_rate
0,health_beauty,21,1233131.72,2,36,8498,142,0.02
1,watches_gifts,12,1166176.98,2,20,5421,61,0.01
2,bed_bath_table,11,1023434.76,1,18,9008,245,0.03
3,sports_leisure,7,954852.55,2,47,7341,170,0.02
4,computers_accessories,20,888724.61,5,35,6405,119,0.02


In [99]:
df_products_performance.to_csv(f'{BI_FOLDER_PATH}bi_product_performance.csv', index=False)

# Monthly Marketplace Table

In [127]:
df_monthly_trend = run_query(
    """
    --- get first order date
    WITH first_order AS (
        SELECT
            c.customer_unique_id,
            MIN(o.order_purchase_timestamp) AS first_order_date
        FROM orders o
        INNER JOIN customers c ON o.customer_id = c.customer_id
        GROUP BY c.customer_unique_id
    ),

    --- customers placing orders per month
    monthly_customers AS (
        SELECT DISTINCT
            o.order_id,
            o.order_status,
            DATE_TRUNC('month', o.order_purchase_timestamp) AS order_month,
            c.customer_unique_id
        FROM orders o
        INNER JOIN customers c ON o.customer_id = c.customer_id
    )

    --- new vs returning customers are placing orders in month
    --- repeat buyer rate per month?
    SELECT 
        mc.order_month,
        COUNT(mc.customer_unique_id) AS total_customers,
        COUNT(CASE 
            WHEN fo.first_order_date < mc.order_month 
            THEN mc.customer_unique_id 
        END) AS repeat_customers,
        ROUND(
            COUNT(DISTINCT CASE 
                WHEN fo.first_order_date < mc.order_month 
                THEN mc.customer_unique_id 
            END)::numeric 
            / COUNT(DISTINCT mc.customer_unique_id),
            2
        ) AS repeat_buyer_rate,
        COUNT(mc.order_id) AS order_count,                                                               -- total orders per month
        SUM(CASE WHEN mc.order_status = 'delivered' THEN oi.price ELSE 0 END) AS gmv        
    FROM monthly_customers mc
    INNER JOIN first_order fo
        ON mc.customer_unique_id = fo.customer_unique_id
    LEFT JOIN order_items oi 
        ON mc.order_id = oi.order_id
    GROUP BY mc.order_month
    ORDER BY mc.order_month;




    """
)

df_monthly_trend.head()



,order_month,total_customers,repeat_customers,repeat_buyer_rate,order_count,gmv
0,2016-09-01,7,0,0.0,7,134.97
1,2016-10-01,379,0,0.0,379,40325.11
2,2016-12-01,1,0,0.0,1,10.90
3,2017-01-01,966,1,0.0,966,111798.36
4,2017-02-01,1998,3,0.0,1998,234223.40


In [128]:
df_monthly_trend['month_num'] = df_monthly_trend['order_month'].dt.month
monthly_repeat_rate_sum = df_monthly_trend.groupby('month_num')['repeat_buyer_rate'].sum()
print(monthly_repeat_rate_sum)


month_num
1     0.02
2     0.02
3     0.02
4     0.03
5     0.04
6     0.04
7     0.04
8     0.04
9     0.66
10    0.77
11    0.02
12    0.02
Name: repeat_buyer_rate, dtype: float64


In [119]:
df_monthly_trend['order_month'].max()

Timestamp('2018-10-01 00:00:00')

In [101]:
df_monthly_trend.to_csv(f'{BI_FOLDER_PATH}bi_monthly_trend.csv', index=False)

In [102]:
df_active_buyers = run_query(
    """
    SELECT 
        c.customer_city,
        DATE_TRUNC('month', o.order_purchase_timestamp) AS order_month,
        COUNT(DISTINCT o.customer_id) AS active_buyers
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.customer_city, order_month
    ORDER BY active_buyers DESC;

    """
)

df_active_buyers.head()

,customer_city,order_month,active_buyers
0,sao paulo,2018-08-01,1308
1,sao paulo,2018-05-01,1222
2,sao paulo,2018-04-01,1166
3,sao paulo,2018-03-01,1151
4,sao paulo,2017-11-01,1118


In [116]:
df_active_buyers.to_csv(f'{BI_FOLDER_PATH}bi_active_buyers.csv', index=False)

# new vs returning customers

In [104]:
df_new_vs_return_customers = run_query(
    """
    --- new vs returning customers placing orders
    SELECT
        c.customer_unique_id,
        CASE 
            WHEN COUNT(customer_unique_id) = 1 
            THEN 'new'
            ELSE 'returning'
        END AS customer_type,
        COUNT(DISTINCT o.customer_id) AS total_num_customers,
        COUNT(o.order_id) AS total_num_orders
    FROM orders o
    INNER JOIN customers c 
        ON o.customer_id = c.customer_id
    GROUP BY customer_unique_id


    """
)

df_new_vs_return_customers.head()

,customer_unique_id,customer_type,total_num_customers,total_num_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,new,1,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,new,1,1
2,0000f46a3911fa3c0805444483337064,new,1,1
3,0000f6ccb0745a6a4b88665a16c9f078,new,1,1
4,0004aac84e0df4da2b147fca70cf8255,new,1,1


In [105]:
df_new_vs_return_customers.to_csv(f'{BI_FOLDER_PATH}bi_new_vs_return_customers.csv', index=False)

# Operational / Cancellation Metrics

What is the order cancellation rate over time?

Are cancellations clustered by seller or region?

Which product categories have the most canceled orders?

In [106]:
# order cancellation rate over time
df_cancellation_metrics_over_time = run_query(
    """
        SELECT
            DATE_TRUNC('month', order_purchase_timestamp) AS month,
            COUNT(*) AS total_orders,
            COUNT(CASE WHEN order_status = 'canceled' THEN 1 END) AS canceled_orders,
            (COUNT(CASE WHEN order_status = 'canceled' THEN 1 END)::numeric / COUNT(*)) * 100 AS cancellation_rate
        FROM orders
        GROUP BY DATE_TRUNC('month', order_purchase_timestamp)
        ORDER BY month;

    """
)

df_cancellation_metrics_over_time.head()

,month,total_orders,canceled_orders,cancellation_rate
0,2016-09-01,4,2,50.000000
1,2016-10-01,324,24,7.407407
2,2016-12-01,1,0,0.000000
3,2017-01-01,800,3,0.375000
4,2017-02-01,1780,17,0.955056


In [107]:
df_cancellation_metrics_over_time.to_csv(f'{BI_FOLDER_PATH}bi_cancellation_metrics_over_time.csv', index=False)

In [108]:
df_cancellation_metrics_by_seller = run_query(
    """
    --- cancellation rate by seller
    WITH canceled_orders AS (
        SELECT 
            s.seller_id,
            s.seller_city,
            s.seller_state,
            COUNT(DISTINCT o.order_id) AS cancel_count
        FROM orders o
        INNER JOIN order_items oi ON o.order_id = oi.order_id
        INNER JOIN sellers s ON oi.seller_id = s.seller_id
        WHERE o.order_status='canceled'
        GROUP BY s.seller_id, s.seller_city, s.seller_state
    ),
    total_orders AS (
        SELECT 
            s.seller_id,
            COUNT(o.order_id) OVER (PARTITION BY s.seller_id) AS total_count,
            DATE_TRUNC('month', o.order_purchase_timestamp) AS order_month
        FROM orders o
        INNER JOIN order_items oi ON o.order_id = oi.order_id
        INNER JOIN sellers s ON oi.seller_id = s.seller_id
    )
    SELECT DISTINCT
        c.seller_id,
        c.seller_city,
        c.seller_state,
        t.total_count AS total_order_count, 
        t.order_month,
        c.cancel_count,
        c.cancel_count::numeric / t.total_count AS cancel_rate
    FROM canceled_orders c
    INNER JOIN total_orders t
        ON t.seller_id = c.seller_id
    
    """
)

df_cancellation_metrics_by_seller.head()

,seller_id,seller_city,seller_state,total_order_count,order_month,cancel_count,cancel_rate
0,001e6ad469a905060d959994f1b41e4f,sao goncalo,RJ,1,2018-08-01,1,1.000000
1,02a2272692e13558373c66db98f05e2e,rio de janeiro,RJ,2,2017-02-01,1,0.500000
2,02a2272692e13558373c66db98f05e2e,rio de janeiro,RJ,2,2017-03-01,1,0.500000
3,0307f7565ff85b299b6e5ef162b1b1a9,sao jose do rio preto,SP,7,2016-10-01,1,0.142857
4,0307f7565ff85b299b6e5ef162b1b1a9,sao jose do rio preto,SP,7,2017-02-01,1,0.142857


In [109]:
df_cancellation_metrics_by_seller.to_csv(f'{BI_FOLDER_PATH}bi_cancellation_metrics_by_seller.csv', index=False)

# Delivery / Logistics

What is the average delivery time?

Are certain sellers or cities experiencing delays?

In [110]:
# we can use this to analyze average delivery time by city and by seller and by order in bi
df_avg_delivery_time = run_query(
    """
        SELECT
            o.order_id,
            oi.order_item_id,
            c.customer_city,
            s.seller_id,
            EXTRACT(EPOCH FROM (o.order_delivered_customer_date - o.order_purchase_timestamp))/86400 AS delivery_days
        FROM orders o
        INNER JOIN order_items oi ON o.order_id = oi.order_id
        INNER JOIN sellers s ON oi.seller_id = s.seller_id
        LEFT JOIN customers c ON o.customer_id = c.customer_id
        WHERE o.order_delivered_customer_date IS NOT NULL
    """
)

df_avg_delivery_time.head()

,order_id,order_item_id,customer_city,seller_id,delivery_days
0,000f25f4d72195062c040b12dce9a18a,1,volta redonda,7c67e1448b00f6e969d365cea6b010ab,15.262951
1,001021efaa8636c29475e7734483457d,1,ivaipora,6560211a19b47992c3666cc44a7e94c0,9.443970
2,0010b2e5201cc5f1ae7e9c6cc8f5bd00,1,rio de janeiro,3504c0cb71d7fa48d967e0e4c94d59d9,11.820694
3,00119ff934e539cf26f92b9ef0cdfed8,1,nova iguacu,c864036feaab8c1659f65ea4faebe1da,10.699421
4,0011d82c4b53e22e84023405fb467e57,1,sao paulo,bfd27a966d91cfaafdb25d076585f0da,10.021285


In [111]:
df_avg_delivery_time.to_csv(f'{BI_FOLDER_PATH}bi_avg_delivery_time.csv', index=False)

In [112]:
df_avg_delivery_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110196 entries, 0 to 110195
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       110196 non-null  object 
 1   order_item_id  110196 non-null  int64  
 2   customer_city  110196 non-null  object 
 3   seller_id      110196 non-null  object 
 4   delivery_days  110196 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ MB


In [113]:
df1 = df_avg_delivery_time.groupby('customer_city')['delivery_days'].mean().reset_index()
df1.sort_values(by='delivery_days', ascending=False).head(10)

,customer_city,delivery_days
2526,novo brasil,148.174502
789,capinzal do norte,109.075029
24,adhemar de barros,97.468507
3245,santa cruz de goias,86.721790
230,arace,86.403944
2202,marcelino vieira,78.182396
3665,serra dos dourados,74.674769
246,aramari,73.660995
1546,humildes,71.520671
2134,machadinho,65.791597


# RFM

In [114]:

df_rfm_segments = run_query(
    """
        WITH customer_rfm AS (
            SELECT
                c.customer_unique_id,
                MAX(o.order_purchase_timestamp) AS last_order_date,       -- for recency
                COUNT(DISTINCT o.order_id) AS frequency,                  -- total orders
                SUM(oi.price) AS monetary                                 -- total GMV
            FROM orders o
            INNER JOIN customers c ON o.customer_id = c.customer_id
            INNER JOIN order_items oi ON o.order_id = oi.order_id
            GROUP BY c.customer_unique_id
        ),
        rfm_with_recency AS (
            SELECT *,
                EXTRACT(DAY FROM (CURRENT_DATE - last_order_date)) AS recency_days
            FROM customer_rfm
        ),

        rfm_scores AS (
            SELECT *,
                NTILE(5) OVER (ORDER BY recency_days ASC) AS recency_score,                     -- smaller recency = higher score
                NTILE(5) OVER (ORDER BY frequency DESC) AS frequency_score,                     -- higher frequency = higher score
                NTILE(5) OVER (ORDER BY monetary DESC) AS monetary_score                        -- higher spend = higher score
            FROM rfm_with_recency
        ),

        rfm_codes AS (
            SELECT *,
                recency_score::text || frequency_score::text || monetary_score::text AS rfm_score
            FROM rfm_scores
        )

        SELECT *,
            CASE
                WHEN rfm_score IN ('555','554','544','545','454','455','445') THEN 'Champions'
                WHEN rfm_score IN ('543','444','435','355','354','345','344','335') THEN 'Loyal'
                WHEN rfm_score IN ('553','551','552','541','542','533','532','531','452','451','442','441','431','453','433','432','423','353','352','351','342','341','333','323') THEN 'Potential Loyalist'
                WHEN rfm_score IN ('512','511','422','421','412','411','311') THEN 'New Customers'
                WHEN rfm_score IN ('525','524','523','522','521','515','514','513','425','424','413','414','415','315','314','313') THEN 'Promising'
                WHEN rfm_score IN ('535','534','443','434','343','334','325','324') THEN 'Need Attention'
                WHEN rfm_score IN ('331','321','312','221','213','231','241','251') THEN 'About To Sleep'
                WHEN rfm_score IN ('155','154','144','214','215','115','114','113') THEN 'Cannot Lose Them'
                WHEN rfm_score IN ('255','254','245','244','253','252','243','242','235','234','225','224','153','152','145','143','142','135','134','133','125','124') THEN 'At Risk'
                WHEN rfm_score IN ('332','322','233','232','223','222','132','123','122','212','211') THEN 'Hibernating Customers'
                ELSE 'Other'
            END AS customer_segment
        FROM rfm_codes;

    """
)

df_rfm_segments.head()

,customer_unique_id,last_order_date,frequency,monetary,recency_days,recency_score,frequency_score,monetary_score,rfm_score,customer_segment
0,ff22e30958c13ffe219db7d711e8f564,2018-09-03 09:06:57,2,290.00,2702.0,1,1,1,111,Other
1,5c58de6fb80e93396e2f35642666b693,2018-08-29 14:18:23,1,418.00,2707.0,1,1,1,111,Other
2,0c6d7218d5f3fa14514fd29865269993,2018-08-29 09:48:09,1,169.80,2707.0,1,2,2,122,Hibernating Customers
3,afbcfd0b9c5233e7ccc73428526fbb52,2018-08-29 08:46:11,1,457.75,2707.0,1,1,1,111,Other
4,b701bebbdf478f5500348f03aff62121,2018-08-29 14:52:00,1,24.90,2707.0,1,3,5,135,At Risk


In [115]:
df_rfm_segments.to_csv(f'{BI_FOLDER_PATH}bi_rfm_segments.csv', index=False)